In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
!pip3 install --upgrade imutils
import cv2
import os

In [ ]:
dataset_info = pd.read_csv("../input/isic-2019/ISIC_2019_Training_GroundTruth.csv")
del dataset_info[dataset_info.columns[-1]]
adress_list=[]
for i in range(1,9):
    adress_list.append(dataset_info[dataset_info[dataset_info.columns[i]]==1].image)
for i in range(1,9):
    os.makedirs(f"Dataset/{i-1}/{i-1}")
for i in range(1,9):
    os.makedirs(f"DatasetAugmented/{i-1}")

In [ ]:
input_size=256
for i in range(0,8):
#     counter=0
    for j in range(0,len(adress_list[i])):
        src_path = '../input/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input/'+ adress_list[i].iloc[j] + '.jpg'
        dst_path = f"Dataset/{i}/{i}/"+adress_list[i].iloc[j] + '.png'
        image = cv2.resize(cv2.imread(src_path),dsize=(input_size,input_size))
        cv2.imwrite(dst_path,image)
        del image
#         counter+=1
#         if counter>9:
#             break
    print(f"category{i} done")

In [ ]:
category=['MEL','NV','BCC','AK','BKL','DF','VASC','SCC']
random_items=[random.randint(0,max) for max in [len(i) for i in adress_list]]
src_path=[]
for i,j in  zip(range(9),random_items):
    src_path.append('../input/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input/'+ adress_list[i].iloc[j] + '.jpg')
plt.figure(figsize=(80,60))
for i,t in zip(range(8),category):
    plt.subplot(2,4,i+1)
    plt.title(t,fontsize=120,fontweight='bold')
    plt.imshow(np.asarray(cv2.resize(plt.imread(src_path[i]),dsize=(400,400))))
    plt.axis('off')
plt.savefig('dataset_diffrent_categories.png', dpi=199)

In [ ]:
from keras.preprocessing import image
for i in range(8):
    original_path=f'Dataset/{i}/'
    agumented_path=f'DatasetAugmented/{i}/'
    step=10#23
    bach_size=int(len(adress_list[i])/step)
    data_generator=image.ImageDataGenerator(brightness_range=[0.2,0.4]).flow_from_directory(original_path,batch_size=bach_size,save_to_dir=agumented_path,save_prefix='b',target_size=(input_size,input_size))
    for i in range(step):
        data_generator.next()
    del data_generator
    data_generator=image.ImageDataGenerator(rotation_range=0.2).flow_from_directory(original_path,batch_size=bach_size,save_to_dir=agumented_path,save_prefix='r',target_size=(input_size,input_size))
    for i in range(step):
        data_generator.next()
    del data_generator
    data_generator=image.ImageDataGenerator(width_shift_range=0.2).flow_from_directory(original_path,batch_size=bach_size,save_to_dir=agumented_path,save_prefix='w',target_size=(input_size,input_size))
    for i in range(step):
        data_generator.next()
    del data_generator
    data_generator=image.ImageDataGenerator(height_shift_range=0.2).flow_from_directory(original_path,batch_size=bach_size,save_to_dir=agumented_path,save_prefix='h',target_size=(input_size,input_size))
    for i in range(step):
        data_generator.next()
    del data_generator
import glob
import shutil
for i in range(8):
    augmented_image_list=glob.glob(f'DatasetAugmented/{i}/*.png')
    for image in augmented_image_list:
        shutil.move(image,f'Dataset/{i}/{i}/')
shutil.rmtree('DatasetAugmented')
print("moved successfuly")
# shutil.make_archive("augmentedDataset", 'zip', 'Dataset/')
# shutil.make_archive("Dataset", 'zip', 'Dataset/') 

In [ ]:
import glob
import shutil
all_images_address=[]
all_image_count=[]
for i in range(8):
    augmented_image_list=glob.glob(f'Dataset/{i}/{i}/*.png')
    all_image_count.append(len(augmented_image_list))
    all_images_address+=augmented_image_list
labels=[]
cat=0
for category in all_image_count:
    for i in range(category):
        labels.append(cat)
    cat+=1
train_address= np.array(all_images_address, dtype=object)
labels=np.expand_dims(np.array(labels),axis=1)
train_data=np.array([np.array(cv2.imread(image))/np.max(np.array(cv2.imread(image))) for image in train_address])
print(labels.shape)
print(train_data.shape)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor as outlier_detector
print(train_data.shape)
print(labels.shape)
model=outlier_detector()
predicts=model.fit_predict(train_data.reshape(train_data.shape[0],-1),labels)
outlier_indicies=np.where(predicts==-1)[0]
train_data=np.delete(train_data, outlier_indicies,axis=0)
labels=np.delete(labels, outlier_indicies)
print(train_data.shape)
print(labels.shape)

In [ ]:
from sklearn.metrics import roc_curve,classification_report,confusion_matrix,auc,accuracy_score
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from keras.models import load_model
from keras.utils import np_utils
from numpy import dstack
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import classification_report
import datetime
def XGB(x_train,y_train,x_test,y_test,members):
    
    def stacked_dataset(members, x_train):
        stackX = None
        for model in members:
            yhat = model.predict(x_train, verbose=0)
            if stackX is None:
                stackX = yhat
            else:
                stackX = dstack((stackX, yhat))
        stackX = stackX.reshape((stackX.shape[0], stackX.shape[1] * stackX.shape[2]))
        return stackX

    def fit_stacked_model(members,x_train,y_train):
        stackedX=stacked_dataset(members,x_train)
        model=XGBClassifier()
        model.fit(stackedX,y_train)
        return model


    def stacked_prediction(members, model, inputX):
        stackedX = stacked_dataset(members, inputX)
        yhat = model.predict(stackedX)
        return yhat
    
    print("entering XGB")
    all_models = list()
    for i in range(2):
        filename = f'./CNN_fold' + str(i + 1) + '.h5'
        model = load_model(filename)
        all_models.append(model)  
    
    
    model = fit_stacked_model(all_models, x_train, y_train)
    yhat = stacked_prediction(all_models, model, x_test)
    acc1 = accuracy_score(y_test, yhat)
    print(f"accuracy is : {acc1}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import tensorflow
import copy

def buildCNN(input_size):
    model=Sequential()
    model.add(Conv2D(256,3,padding='same',activation='relu',strides=2, input_shape=(input_size,input_size,3)))
    model.add(Dropout(0.3))
    model.add(Conv2D(512,3,padding='same',activation='relu',strides=2))
    model.add(Dropout(0.3))
    model.add(Conv2D(256,3,padding='same',activation='relu',strides=2))
    model.add(Dropout(0.3))
    model.add(Conv2D(256,3,padding='same',activation='relu',strides=1))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='softmax'))
    
    model.compile(loss=tensorflow.keras.losses.sparse_categorical_crossentropy,
                optimizer="Adam",
                metrics=['accuracy'])
    return model

# def train_model(train_batch_adresses,labels,ep,batch_size,model1,model2):
#     labels= copy.deepcopy(labels)
#     np.random.shuffle(labels)
#     for j in range(ep):
#         for i in range(int(train_batch_adresses.shape[0]/batch_size)-1):
#                 X= np.array([np.array(cv2.imread(image)) for image in train_batch_adresses[i*batch_size:(i+1)*batch_size]])
#                 Y= labels[i*batch_size:(i+1)*batch_size]
#                 model1.fit(X,Y,epochs=1,verbose=0)
#                 model2.fit(X,Y,epochs=1,verbose=0)
#                 del X

In [ ]:
from sklearn.metrics import accuracy_score
def XGB(x_train,y_train,x_test,y_test):
    
    def load_all_models(n_models):
        all_models = list()
        for i in range(n_models):
            filename = f'./CNN_fold' + str(i + 1) + '.h5'
            model = load_model(filename)
            all_models.append(model)
            print('>loaded %s' % filename)
        return all_models
    
    def stacked_dataset(members, inputX):
        stackX = None
        for model in members:
            yhat = model.predict(inputX, verbose=0)
            if stackX is None:
                stackX = yhat
            else:
                stackX = dstack((stackX, yhat))
        stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
        return stackX

    def fit_stacked_model(members,inputX,inputy):
        stackedX=stacked_dataset(members,inputX)
        model=XGBClassifier()
        start=datetime.datetime.now()
        model.fit(stackedX,inputy)
        end=datetime.datetime.now()
        training_time=end-start
        return model,training_time

    def stacked_prediction(members, model, inputX):
        stackedX = stacked_dataset(members, inputX)
        yhat = model.predict(stackedX)
        return yhat

    trainX=x_train
    testX=x_test
    trainy=y_train
    testy=y_test
    
    n_members = 2
    members = load_all_models(n_members)
    print('Loaded %d models' % len(members))
    # evaluate standalone models on test dataset
#     for model in members:
#         testy_enc = testy #testy_enc = np_utils.to_categorical(testy)
#         loss, acc = model.evaluate(testX, testy_enc, verbose=0)
    model,training_time = fit_stacked_model(members, trainX, trainy)
    yhat = stacked_prediction(members, model, testX)
    predicts=yhat
    actuals=testy
    acc1 = accuracy_score(actuals, predicts)
    print(f"xgb acc:{acc1}")


In [ ]:
folds_count=10
kfold = KFold(n_splits=folds_count, shuffle=True,random_state=None)
for train, test in kfold.split(train_data, labels):
    models=[]
    inner_kfold = KFold(2,shuffle=True,random_state=None)
    x_data=train_data[train]
    y_data=labels[train]
    counter=1
    for t, v in inner_kfold.split(x_data,y_data):
        inner_x_train=x_data[t]
        inner_y_train=y_data[t]
        inner_x_test=x_data[v]
        inner_y_test=y_data[v]
        model=buildCNN(input_size)
        model.fit(inner_x_train,inner_y_train,epochs=60,batch_size=256,validation_data=(inner_x_test,inner_y_test),verbose=0)#,verbose=0
        print(f"inner result:{model.evaluate(inner_x_test,inner_y_test, verbose=0)}")
        model.save(f'./CNN_fold{counter}.h5')
        models.append(model)
        counter+=1
    print(f"outer cNN results:{models[0].evaluate(train_data[test],labels[test], verbose=0),models[0].evaluate(train_data[test],labels[test], verbose=0)}")
#     XGB(inner_x_train,inner_y_train,train_data[test],labels[test],models)
    XGB(train_data[train],labels[train],train_data[test],labels[test])

    
#     XGB(inner_x_test,inner_y_test,train_data[test],labels[test])

In [ ]:
#lasti

In [ ]:
    train_model(train_address[train],labels[train],epochs,int((train_address[train].shape[0]/100)),model1,model2)
    model1_evaluate_results=model1.evaluate(train_data[test],labels[test], verbose=0)
    print(f"M1:{model1_evaluate_results}")
    XGB(train_data[train],labels[train],train_data[test],labels[test],models)
    
    test_x= np.array([np.array(cv2.imread(image)) for image in train_address[test]])
    model1_evaluate_results=model1.evaluate(test_x, labels[test], verbose=0)
    model2_evaluate_results=model2.evaluate(test_x, labels[test], verbose=0)
    print(f"M1:{after_evaluate_results}      M2:{after_evaluate_results}")
    del test_x
shutil.rmtree('Dataset')
print("datasetCleaned")

In [ ]:

fold_number=1
kfold=KFold(10,shuffle=True,random_state=None)
for train,test in kfold.split(train_data, labels):
    x_data=x[train]
    y_data=y[train]
    x_test=x[test]
    test_labels=y[test]

    counter=1
    n_epch=30

    from sklearn.model_selection import KFold
    kfold = KFold(2,shuffle=True,random_state=None)
    for train, valid in kfold.split(x_data,y_data):
        x_train=x_data[train]
        x_valid=x_data[valid]
        train_labels=y_data[train]
        valid_labels=y_data[valid]

        from keras.utils import np_utils
        y_train=keras.utils.to_categorical(train_labels)
        y_test=np_utils.to_categorical(test_labels)
        y_valid=np_utils.to_categorical(valid_labels)

        model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
        net_history=model.fit(x_train, y_train, batch_size=256, epochs=n_epch ,validation_data=(x_valid,y_valid),callbacks=[calback])
    import XGBoost2
    XGBoost2.XGB(x_train,x_test,train_labels,test_labels,fold_number)

    fold_number+=1


print('Successfully was done...!')